In [2]:
import pandas as pd
import numpy as np

In [42]:
data = pd.read_csv('data/data_ready.csv')
# Get the Y labels
map_code = {'non-graine':0, 'graine':1}
data['label'] = data['sentiment'].map(map_code)

In [37]:
emb = {}
emb['tfidf'] = np.load('embeddings/tfidf.npy')
emb['bert'] = np.load('embeddings/bert.npy')
emb['features'] = np.load('embeddings/features.npy')
emb['profile_pictures'] = np.load('embeddings/full_profile_pictures.npy')

In [38]:
full_list = (emb['tfidf'], emb['profile_pictures'], emb['features'], emb['bert'])

X = np.concatenate(full_list, axis=1)

In [39]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, euclidean_distances
from sklearn.svm import SVC


def triangular_kernel(X, Y):
    return 1 - abs(euclidean_distances(X, Y))


classifiers = {
    "SVM_triangular_kernel": SVC(kernel=triangular_kernel, C=3),
    "SVM_RBF_kernel": SVC(),
}



In [40]:
X = emb['bert']
y = list(data.label)

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=27)

clf = classifiers['SVM_triangular_kernel']
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(pd.DataFrame(y_pred).value_counts())

precision, recall, f1, _ = precision_recall_fscore_support(
                y_test, y_pred, pos_label=1, average="binary"
            )
print(precision, recall, f1)

0    1253
1       4
dtype: int64
0.5 0.025 0.047619047619047616
